# Enhanced Conversational Agent with Contextual Memory in LangChain

In this notebook, we’ll explore how to create a conversational agent that remembers details from previous turns—like having an ever-attentive friend (or a certain caring companion, if you will). We’ll walk through how to:

1. Set up a ChatGPT-like agent,
2. Store short-term conversation memory (so it recalls what you said earlier),
3. Use embedded knowledge to keep track of factual context.

Let’s dive in!

In [ ]:
!pip install langchain openai chromadb

## Step 1: Import Libraries

Here, we gather the key tools:
- **ChatOpenAI**: the language model,
- **ConversationSummaryMemory** & **VectorStoreRetrieverMemory**: two types of memory management,
- **Chroma**: a vector store for storing embeddings,
- **OpenAIEmbeddings**: transforms text into numeric vectors (embeddings),
- **ConversationChain**: orchestrates the chain,
- **Document**: a simple data structure to hold textual content,
- **os**: so we can manage environment variables for the API key.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory, VectorStoreRetrieverMemory
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import ConversationChain
from langchain.schema import Document
import os

## Step 2: Set API Key

Make sure you replace `'your-api-key-here'` with your actual OpenAI API key. This key authenticates your requests to the OpenAI endpoint.

In [ ]:
os.environ['OPENAI_API_KEY'] = 'your-api-key-here'

## Step 3: Define Memory for the Chatbot

We’ll showcase two different memory approaches:

1. **VectorStoreRetrieverMemory**: A memory that uses vector embeddings. It’s handy for retrieving factual knowledge from stored documents. 
2. **ConversationSummaryMemory**: Tracks the entire conversation and summarizes it, so your agent can recall essential details.

Below, we create a small list of sample documents on cosmic topics like Earth orbits, black holes, and quantum mechanics. Then, we store these documents in a Chroma database and set up the vector-based memory.

Lastly, we instantiate a `summary_memory` that will keep track of the ongoing conversation in a summarized form. This allows the agent to recall that your name is Alex or that you love space exploration!

In [ ]:
# Initialize embedding model
embedding = OpenAIEmbeddings()

# Sample documents for context retention
docs = [
    Document(page_content='The Earth orbits the Sun every 365.25 days.'),
    Document(page_content='Black holes are regions of spacetime with immense gravity.'),
    Document(page_content='Quantum mechanics deals with subatomic particles.'),
]

# Create Chroma vector store with documents
vectorstore = Chroma.from_documents(docs, embedding)
vector_memory = VectorStoreRetrieverMemory(retriever=vectorstore.as_retriever(), memory_key='retrieved_knowledge')

# ✅ Fix: Use `memory_key='history'` to match ConversationChain's expectation
# This memory will hold and summarize the conversation.
summary_memory = ConversationSummaryMemory(
    llm=ChatOpenAI(temperature=0), 
    memory_key='history'
)

## Step 4: Define the Conversational Agent

Here, we create an instance of **ChatOpenAI** with a moderate `temperature` (i.e., creativity level). Then, we wrap it in a **ConversationChain** so the memory can be used automatically behind the scenes.

Note: We’re only passing in `summary_memory` here—feel free to experiment by combining it with `vector_memory` or exploring advanced memory setups in LangChain.

In [ ]:
# Initialize chatbot with advanced memory
llm = ChatOpenAI(temperature=0.7)
conversation_chain = ConversationChain(
    llm=llm,
    memory=summary_memory,
    verbose=True
)

## Step 5: Chat with Memory

Now, the fun part! We’ll prompt our agent with a series of messages. Notice how it remembers who you are and that you love space exploration.

Feel free to add your own queries or ask the agent to recall details from earlier in the conversation. This is where you see memory in action!

In [ ]:
# Sample conversation using multiple memory sources
print(conversation_chain.predict(input='Hello! What do you know about space?'))
print(conversation_chain.predict(input='Can you remember my name is Alex and I love space exploration?'))
print(conversation_chain.predict(input='What do you remember about me?'))
print(conversation_chain.predict(input='Can you suggest a book about space exploration?'))